# Code for TSP solving with QAOA

In [6]:
import os
import random
import time
from collections import deque
from itertools import permutations
import operator

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib
matplotlib.use("TkAgg")  # Set non-interactive backend
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.optimize import minimize

import unittest

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import QAOAAnsatz, TwoLocal
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

import qiskit_aer as Aer
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error

from qiskit.primitives import Sampler, Estimator, StatevectorSampler

from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    SamplerV2,
    SamplerV2 as Sampler2,  # optional, remove if not needed
    Session,
    EstimatorV2 as Estimator3,  # optional, rename or remove as needed
)

from qiskit_algorithms import (
    QAOA,
    VQE,
    SamplingVQE,
    NumPyMinimumEigensolver,
)
from qiskit_algorithms.optimizers import (
    COBYLA,
    SPSA,
    ADAM,
)
from qiskit_algorithms.utils import algorithm_globals

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import Tsp
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import (
    QuadraticProgramToQubo,
    InequalityToEquality,
)

from docplex.mp.model import Model

In [7]:
import unittest
import qiskit
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
from qiskit_ibm_runtime import SamplerV2 as SamplerV22
import numpy as np
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as SamplerV2
from qiskit.primitives import Sampler, Estimator
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz
import qiskit_aer as Aer
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from scipy.optimize import minimize
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.converters import InequalityToEquality
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter
import matplotlib

# matplotlib.use("TkAgg")  # Set non-interactive backend
import matplotlib.pyplot as plt
from qiskit import transpile
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import os

## Define Problem

In [8]:
class TSP_Problem:
    def __init__(self, num_cities):
        self.n = num_cities
        """self.tsp = Tsp.create_random_instance(self.n, seed=123)"""
        self.tsp = Tsp.create_random_instance(self.n)
        self.adj_matrix = nx.to_numpy_array(self.tsp.graph)

        self.convert()

    def convert(self):
        self.qp = self.tsp.to_quadratic_program()
        conv = QuadraticProgramToQubo()
        self.qubo = conv.convert(self.qp)
        self.qubitOp, self.offset = self.qubo.to_ising()

    def draw_graph(self, G, colors, pos):
        plt.figure(figsize=(10, 10))
        ax = plt.axes(frameon=True)

        # Draw edges
        nx.draw_networkx_edges(G, pos, width=2, alpha=0.5)

        # Draw nodes
        nx.draw_networkx_nodes(G, pos, node_color=colors, node_size=800, alpha=0.8)

        # Draw node labels
        nx.draw_networkx_labels(G, pos, font_size=14, font_weight="bold")

        # Draw edge labelsCode/TSP/Qiskit.ipynb
        # Remove axis
        plt.axis("off")

        # Add some padding around the graph
        plt.tight_layout()

## Solving the problem

In [9]:
meo = MinimumEigenOptimizer(QAOA(reps=3, sampler=Sampler(), optimizer=COBYLA(maxiter=100)))
# meo = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # CplexOptimizer() # to solve QUBOs

qp = TSP_Problem(4).qp
result_qaoa = meo.solve(qp)

/tmp/ipykernel_46499/2893032751.py:1: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  meo = MinimumEigenOptimizer(QAOA(reps=3, sampler=Sampler(), optimizer=COBYLA(maxiter=100)))


In [12]:
print(result_qaoa)

fval=205.0, x_0_0=0.0, x_0_1=0.0, x_0_2=0.0, x_0_3=1.0, x_1_0=0.0, x_1_1=0.0, x_1_2=1.0, x_1_3=0.0, x_2_0=0.0, x_2_1=1.0, x_2_2=0.0, x_2_3=0.0, x_3_0=1.0, x_3_1=0.0, x_3_2=0.0, x_3_3=0.0, status=SUCCESS
